# Intro

This is my progress in the implementation of Hermite mappings for the quadrature representation of cell integrals. See hermite.tm for the math.

The notebook [elements](elements.ipynb) contains some nomenclature, including definitions of: *finite element*, *dof* (and its action on functions), *intermediate representation*.

There is also a list of [related commits and PRs](elements.ipynb#Related-commits-and-PRs).


# Form compilation with FFC

The function `ffc.compile_form(F)` produces a  c++ header with the code which is later wrapped into a python API for using the form `F`. The generated header contains classes with:

* Finite elements.
* Local to global DOF maps.
* **Cell integral computation:** an implementation of the interface defined in the abstract class `ufc::cell_integral`. This is what interests us here.
* The form itself.

The compiler proceeds in the 5 stages described §11.4 of the FEniCS book. Relevant to us are the second, the intermediate representation, and the fourth ones, the generation of the code. The formers happens mainly in `quadraturerepresentation.py` and the latter in `quadraturegenerator.py`, but the translation from UFL into code, where the core of e.g. the basis functions transformation happens is `quadraturetransformerbase.py`.


## A simple example

Begin with the easiest:

Define $\Omega = [0,1]$ with a mesh of one cell. The space $V$ will be the four dimensional space of polynomials of degree up to three, defined over $\Omega$. The basis chosen will be dual to the Hermite degrees of freedom in the sense described in [hermite.tm](hermite.tm) (basically: two point evaluations and two derivative evaluations at the nodes $v_0 = 0$ and $v_1 = 1$).

We now assemble the mass matrix for the form

$$ \int_\Omega  u \  v \ \mathrm{d}x$$

which amounts to computing the quantity

$$ \int_\Omega \phi_i \phi_j \ \mathrm{d}x$$

for all basis functions in $V$. Recall that the interface specification for `ufc::cell_integral::tabulate_tensor()` declares that it should compute

$$ \int_T \phi_i(x)\ \mathrm{d}x 
 = \int_T \hat{\phi}_i(F^{-1}(x))\ \mathrm{d}x 
 = \int_\hat{T} \hat{\phi}_i(\hat{x}) J\ \mathrm{d}\hat{x}. $$
 
Our goal is to perform the computation both manually and with the code generated by FFC via dolfin's `assemble()` and compare the results. As an additional check, we can test our procedure with Lagrange elements.

Further tests with this code are for the forms

$$ \int_\Omega \nabla u \nabla v \ \mathrm{d}x$$

and

$$ \int_\Omega \Delta u \Delta v \ \mathrm{d}x$$

In [ ]:
from __future__ import print_function

from dolfin import *
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline
import ffc

#from ffc.log import add_logfile, set_level, DEBUG
#set_level(DEBUG)
#add_logfile("/tmp/fenics.log")

import nbimporter
from elements import evaluate_shape_functions_reference

parameters['form_compiler']['no-evaluate_basis_derivatives'] = False
# FIXME: why do I need such a high degree? Do I?
fc_params = {'representation': 'quadrature', 'quadrature_degree': 7}

In [ ]:
mesh = UnitIntervalMesh(3)
V = FunctionSpace(mesh, 'Hermite', 3)

u = TrialFunction(V)
v = TestFunction(V)

a = u.dx(0).dx(0)*v.dx(0).dx(0)*dx
#a = u.dx(0)*v.dx(0)*dx
#a = u*v*dx
A = assemble(a, form_compiler_parameters=fc_params)
with open("/tmp/mass_%s1D.h" % V.ufl_element().family(), "w") as fd:
    out = ffc.compile_form(a, "mass", parameters=fc_params)
    fd.write(out[0])

print("Matrix assembled with FFC-generated code:\n%s" % A.array().round(3))

## One cell integral

The following is a translation into Python of the code produced with `ffc.compile_form()`.

In [ ]:
# quadrature points
#WP = np.array([0.046910077030668, 0.230765344947158, 0.5, 0.769234655052841, 0.953089922969332])
WP = np.array([0.0694318442029737, 0.330009478207572, 0.669990521792428, 0.930568155797026])

# quadrature weights:
#WW = np.array([0.118463442528095, 0.239314335249683, 0.284444444444444, 0.239314335249683, 0.118463442528095])
WW = np.array([0.173927422568727, 0.326072577431273, 0.326072577431273, 0.173927422568727])

# Values of basis functions at quadrature points:
FE0, FE0_DD = evaluate_shape_functions_reference(V.element(), WP, num_derivatives=2)

# Just use the values of the shape functions for the form u*v*dx
#FUN = FE0.T

###########
# EDIT ME: The shape of the returned derivatives is a bit weird
# (dim, num_derivs, num_points) and needs massaging. Also, many of 
# the entries are set to zero and we need to pick the right ones... (?)

# First derivatives for the form u.dx(0)*v.dx(0)*dx
#FUN = FE0_DD.reshape((4, len(WP))).T

# Second derivatives for the form u.dx(0).dx(0)*v.dx(0).dx(0)*dx
FUN = np.zeros((4, len(WP)))
FUN[0:2] = FE0_DD[0,:]
FUN[2:4] = FE0_DD[1,:]
FUN = FUN.T

print("Values of basis functions at quadrature points:\n%s" % FUN)

**Check for Hermite elements:**

Values of basis functions at quadrature points in `tabulate_tensor()`.

For the form `u*v`:

    static const double FE0[4][4] = \
    {{0.986207088460911, 0.0601249979387161, 0.0137929115390892, -0.00448606527483159},
    {0.745161426118203, 0.148137063413256, 0.254838573881797, -0.0729661590874814},
    {0.254838573881795, 0.0729661590874805, 0.745161426118205, -0.148137063413257},
    {0.0137929115390886, 0.00448606527483117, 0.986207088460911, -0.0601249979387162}};

For the form `u.dx(0)*v.dx(0)`:

    static const double FE0_D1[4][4] = \
    {{-0.387666379281288, 0.736734966156382, 0.387666379281288, -0.12440134543767},
    {-1.32661933500443, 0.00668085429021271, 1.32661933500443, -0.333300189294643},
    {-1.32661933500443, -0.333300189294642, 1.32661933500443, 0.00668085429021441},
    {-0.387666379281282, -0.124401345437667, 0.387666379281282, 0.736734966156384}};
    
For the form `u.dx(0).dx(0)*v.dx(0).dx(0)`:
  
    static const double FE0_D2[4][4] = \
    {{-5.16681786956432, -3.58340893478216, 5.16681786956432, -1.58340893478216},
    {-2.03988626150914, -2.01994313075457, 2.03988626150914, -0.0199431307545674},
    {2.03988626150915, 0.0199431307545761, -2.03988626150915, 2.01994313075457},
    {5.16681786956434, 1.58340893478217, -5.16681786956434, 3.58340893478216}};

In [ ]:
# Jacobian for a mesh of constant cell size
det = 1./ V.mesh().num_cells()
idet = V.mesh().num_cells()

# Hermite trafo matrix
H = np.array([1., 0., 0., det])

dim = V.element().space_dimension()
M = np.zeros((dim, dim))
if V.ufl_element().family() == 'Lagrange':
    for ip,j,k in np.ndindex(len(WP), M.shape[0], M.shape[1]):
        M[j,k] += FE0[ip][j]*FE0[ip][k]*WW[ip]*det
elif V.ufl_element().family() == 'Hermite':
    for ip,j,k in np.ndindex(len(WP), M.shape[0], M.shape[1]):
#        print("M[%d,%d] += ((FUN[ip][%d]*H[%d] + FUN[ip][%d]*H[%d])*(FUN[ip][%d]*H[%d] + FUN[ip][%d]*H[%d]))*WW[ip]*det" %
#              (j, k,
#               2*(j/2)+0, 2 * (j % 2) + 0, 2*(j/2)+1, 2 * (j % 2) + 1,
#               2*(k/2)+0, 2 * (k % 2) + 0, 2*(k/2)+1, 2 * (k % 2) + 1))
        M[j,k] += (idet**2*((FUN[ip][2*(j/2)+0])*(H[2 * (j % 2) + 0]) 
                    + (FUN[ip][2*(j/2)+1])*(H[2 * (j % 2) + 1]))*
                   idet**2*((FUN[ip][2*(k/2)+0])*(H[2 * (k % 2) + 0]) 
                    + (FUN[ip][2*(k/2)+1])*(H[2 * (k % 2) + 1])))*WW[ip]*det

#print("Matrix assembled with FFC-generated code:\n%s" % A.array().round(3))
print("FFC-like computation for one cell with quadrature rule:\n %s" % M.round(3))

Note that some of the entries have been permuted wrt. the assembled matrix, where the ordering is given by the local-to-global dof mapping. We will need to take this into account later when we manually assemble everything.


But first note the following. If we compute the same integral using the trapezoidal rule as a check, we will have some entries off by a factor equal to 1/num_cells due to the fact that we are not applying the additional Hermite trafo to the derivatives. So this is basically useless:

In [ ]:
B = np.zeros_like(M)
xx = np.linspace(0, 1, 100)
shapes, derivs = evaluate_shape_functions_reference(V.element(), xx, num_derivatives=1)
#fun = shapes
fun = derivs.reshape((4, len(xx)))
##########
# EDIT ME:
#fun = np.zeros((4, len(xx)))
#fun[0] = derivs[0,0]
#fun[1] = derivs[0,1]
#fun[2] = derivs[1,0]
#fun[3] = derivs[1,1]


##########
for i,j in np.ndindex(B.shape):
    B[i,j] = np.trapz(det*fun[i]*fun[j], xx)

print("Cell integral with trapezoidal rule:\n%s\n" % B.round(3))
print("Trapezoidal - quadrature: %.1f%% error" % (100*np.linalg.norm(B-M, np.inf)/np.linalg.norm(B, np.inf)))

## Mass matrix assembly

The preceding cell integrals need to be assembled into a global mass matrix in order to compare the computations with those done by dolfin. In the case of only one cell in the mesh, we only need to reorder the entries according to the local-to-global dof map. What matters is then the difference between this matrix and the one computed by dolfin:

In [ ]:
Aa = A.array()
Ah = np.zeros_like(Aa)
dm = V.dofmap()
for cell in range(V.mesh().num_cells()):
    l2g = dm.cell_dofs(cell)   # local to global mapping for cell 
    for i,j in np.ndindex(M.shape):
        Ah[l2g[i], l2g[j]] += M[i,j]
#print("Manual assembly:\n%s\n" % Ah.round(3))
#print("Dolfin's assembly:\n%s\n" % Aa.round(3))
print("Error = %.2f%%" % (100*np.linalg.norm(Aa - Ah, np.inf)/np.linalg.norm(Ah, np.inf)))

# Integration of expressions

blah blah...

In [ ]:
grid_sizes = [32, 64, 128, 256]
p_vals = range(1, 4)
results_her = np.zeros(len(grid_sizes))
results_lag = np.zeros((len(grid_sizes), len(p_vals)))

for i, n in enumerate(grid_sizes):
    mesh = UnitSquareMesh(n, n)
    V = FunctionSpace(mesh, "Hermite", 3)
    W = [None] + [FunctionSpace(mesh, "Lagrange", p) for p in p_vals]
    f = Expression("x[0]*x[0]*x[1]*x[1]", degree=3)
    v = project(f, V)
    w = [None] + [project(f, W[p]) for p in p_vals]
    exact = 1./9.
    results_her[i] = assemble(v*dx)
    results_lag[i] = np.array([assemble(w[p]*dx) for p in p_vals])

In [ ]:
pl.figure(figsize=(10, 10))
pl.plot(grid_sizes, np.log(results_her - exact), label="H3")
for p in p_vals:
    pl.plot(grid_sizes, np.log(results_lag[:,p-1] - exact), label="L%d" % p)
pl.title("Log error")
pl.legend()
_ = pl.show()

# What is going on here?

Check this:

In [ ]:
from dolfin import *
import autograd.numpy as np
import nbimporter
from interpolation import interpolate_hermite
from utils import make_derivatives

def find_bubble_dofs(V):
    assert V.ufl_element().family().lower() == 'hermite', "duh"
    bubble_dofs = set()
    dm = V.dofmap()
    for i in range(V.mesh().num_cells()):
        bubble_dofs.add(dm.cell_dofs(i)[-1])
    return list(bubble_dofs)

V = FunctionSpace(UnitSquareMesh(1,1), "Hermite", 3)
bbdofs = find_bubble_dofs(V)

u = TrialFunction(V)
v = TestFunction(V)
a = u*v*dx
A = assemble(a)
print(A.array()[bbdofs,bbdofs])

from shape_functions import hermite_shapes_2d_physical

v1, v2, v3 = Cell(V.mesh(), 0).get_vertex_coordinates().reshape((-1,2))
phi = hermite_shapes_2d_physical(v1, v2, v3)

@make_derivatives
def phi13(x,y):
    return phi[9](np.array([x,y]))

f = Expression("27 * (x[1]*(1-x[0]) - x[1]*x[1]*(1-x[0]) - x[1]*(1-x[0])*(1-x[0]))",
               degree=3)
funp = project(f, V)
funi = interpolate_hermite(phi13, V)
funm = Function(V)
tmp = np.zeros(V.dim())
tmp[13] = 1.   # dof 13 corresponds to phi9
funm.vector().set_local(tmp)

print("Exact integral: -2.25")
print("Projected: %f" % assemble(funp*dx))
print("Interpolated: %f" % assemble(funi*dx))
print("Manual: %f" % assemble(funm*dx))

assemble(funm*dx, form_compiler_parameters={'representation':'quadrature'})

Try projecting using the $H^2$ scalar product:

In [ ]:
f = Expression("27 * (x[1]*(1-x[0]) - x[1]*x[1]*(1-x[0]) - x[1]*(1-x[0])*(1-x[0]))",
               element=V.ufl_element())
def innerH2(u,v):
    return inner(u,v)*dx +\
           inner(grad(u),grad(v))*dx +\
           inner(grad(grad(u)),grad(grad(v)))*dx

u = TrialFunction(V)
v = TestFunction(V)
a = innerH2(u,v)
L = innerH2(f,v)

u = Function(V)
solve(a == L, u)

In [ ]:
fpa = funp.vector().array()
fia = funi.vector().array()

print("%s\n%s\n%s" % (fpa.round(2), fia.round(2), V.dofmap().cell_dofs(1)))

difs = np.where(~np.isclose(fpa, fia, atol=1e-6))[0]
print("%s\n%s\n%s" %(difs, fpa[difs].round(2), fia[difs].round(2)))

from boundary import plot_dofs
plot_dofs(V, difs)

In [ ]:
print(np.where(np.abs(fpa) > 1e-6)[0], np.where(np.abs(fia) > 1e-6)[0])
print(V.dofmap().cell_dofs(0))

In [ ]:
pl.figure(figsize=(10,4))
pl.subplot(1,2,1)
plot_dofs(V, np.where(np.abs(fpa) > 1e-6)[0], color="red")
pl.subplot(1,2,2)
plot_dofs(V, np.where(np.abs(fia) > 1e-6)[0], color="blue")

# Yet another test for integrals of shape functions

A quick test of the integrals of compiled element basis functions (`evaluate_shape_functions_mesh()` invokes FFC's compiled methods)

In [ ]:
from dolfin import *
import matplotlib.pyplot as pl
%matplotlib inline

import numpy as np
np.set_printoptions(precision=2)

from scipy.integrate import dblquad

import nbimporter
from boundary import *
from elements import TODO_evaluate_shape_functions_mesh
from shape_functions import compute_trafo, pt

In [ ]:
V = FunctionSpace(UnitSquareMesh(1, 1), 'Hermite', 3)
v = TestFunction(V)
A = assemble(v*dx)

Aa = A.array()

dm = V.dofmap()
dofs0, dofs1 = dm.cell_dofs(0), dm.cell_dofs(1)

In [ ]:
def phiwrap(i, quadrature=True):
    """Just a hack"""
    def hack(x,y):
        global V
        pt = np.array([[x,y]], dtype=np.float)
        #print("Eval at %s" % pt)
        zz = TODO_evaluate_shape_functions_mesh(V, pt)
        return zz[i][0]
    
    def reverse_args(f):
        def fun(x,y):
            return f(y,x)
        return fun

    F = compute_trafo(pt(0., 0.), pt(1., 0.), pt(1., 1))
    def pack_args(f):
        def fun(X):
            x = F(X)
            return f(min(x[0],1.), min(x[1],1.))
        return fun
    
    return reverse_args(hack) if quadrature else pack_args(hack)

In [ ]:
integrals = np.array([dblquad(phiwrap(dof), 0, 1, lambda x:0., lambda x:x)[0]
                      for dof in range(10)])

Notice how the values in the assembled vector corresponding to dofs shared among both cells are doubled with respect to the values of one cell integral. 👍🏼

In [ ]:
print("%s\n%s " % (integrals, Aa[dofs0]))

plot_dofs(V, dofs0)

A final plot of the basis functions. Notice how two of the Hermite shapes are different from those defined over the reference triangle. Remember that this is because of the Hermite trafo, which for this Jacobian ([1,1],[0,1]) implies that $\phi_4$ and $\phi_7$ are actually the sums $\hat{\phi}_4 + \hat{\phi}_5$ and $\hat{\phi}_7 + \hat{\phi}_8$ respectively 

In [ ]:
from shape_functions import plot_2dshapes

plot_2dshapes([phiwrap(i, quadrature=False) for i in range(10)])